## <a class="anchor" id="main">FoCS Lab</a>
### Contributors: Marco Distrutti, Santosh Anand

* [Normalization](#normalize)
* [Duration Field](#duration)
* [Search lenders](#search-lenders)
* [Overall amount](#everall-amount)
* [Overall percentage](#everall-percentage)


In [1]:
import pandas as pd
import numpy as np
import itertools

df_loan_lenders = pd.read_csv('kiva-kaggle/loans_lenders.csv')
raws = df_loan_lenders

In [2]:
loan_lenders = df_loan_lenders
df_loan_lenders

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
1387427,678999,"michael43411218, carol5987, gooddogg1, chris41..."
1387428,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
1387429,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
1387430,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."


## 1 <a class="anchor" id="normalize">Normalization</a>
#### Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.
[⇑ index](#main)

Using **Vectorized** operations our computations are extremly faster then looping over the dataset because Vectorized operations are heavily implemented in **C** procedures. So the strategy is to create both vectors (loan_id and lender) separatly and loan_id is generated by repeating id with vectorized products.

In [3]:
%%time

#FIRST AXIS
lenders = [lender for lenders in df_loan_lenders['lenders'] for lender in lenders.split(',')]

#SECOND AXIS - using vectorized operations we boost the performance using internal opimized C procedures
loan_ids = [loan_id for loan_id in df_loan_lenders['loan_id']]
cardinality = [len(lenders.split(", ")) for lenders in df_loan_lenders['lenders']]
#create a new list with repeated ids for each loan in the same original row
flatted_ids = list(itertools.chain(*[[loan_ids[i]] * cardinality[i] for i in range(0, len(loan_ids))]))

#DATAFRAME
df_loan_lenders = pd.DataFrame({'loan_id':flatted_ids, 'lender':lenders})

Wall time: 17.2 s


In [4]:
#More then 28 milions of records
df_loan_lenders

,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175
